# Path

Figure out which team was played for first
* Futbin is lower player_id
* Understat is earlier first date in matches or second team in comma list

In [143]:
import numpy as np
import pandas as pd

futbin = pd.read_csv("csv files/futbin_2022.csv")
ustat = pd.read_csv("csv files/ustat_2022.csv")

fix_club_ids = pd.read_csv("data repair csvs/fix_club_ids.csv")
fix_name_ids = pd.read_csv("data repair csvs/fix_name_ids.csv")

In [144]:
futbin = futbin.rename(columns={"name": "player_name", "club": "team_title"})

In [145]:
ustat.team_title = ustat.team_title.str.split(',')
ustat = ustat.explode('team_title').reset_index(drop=True)

## Match name to id

In [146]:
db_id_dict = {}

names = futbin.player_name
dobs = futbin.dob

for i in range(len(names)):
    cur = names[i]
    if cur in db_id_dict:
        db_id_dict[cur] += 1
    else:
        db_id_dict[cur] = 1

db_id_dict

{'Matt Turner': 1,
 'William Saliba': 2,
 'Julián Álvarez': 1,
 'Luis Sinisterra': 1,
 'Rasmus Kristensen': 1,
 'Cheick Doucouré': 2,
 'Fraser Forster': 2,
 'Brandon Williams': 2,
 'Dan Burn': 2,
 'Calum Chambers': 2,
 'Alex Král': 1,
 'Enock Mwepu': 1,
 'Imran Louza': 1,
 'Max Aarons': 1,
 'Alexis Mac Allister': 1,
 'Frank Onyeka': 1,
 'Joseph Willock': 1,
 'Juan Camilo Hernández': 1,
 'Jean-Philippe Mateta': 1,
 'Yoane Wissa': 1,
 'Odsonne Edouard': 1,
 'Takehiro Tomiyasu': 1,
 'Tom Davies': 1,
 'Hee Chan Hwang': 1,
 'Mahmoud Hassan': 1,
 'Ainsley Maitland-Niles': 2,
 'Todd Cantwell': 1,
 'Connor Roberts': 1,
 'Kristoffer Ajer': 1,
 'Marvelous Nakamba': 1,
 'Jaïro Riedewald': 1,
 'Demarai Gray': 1,
 'Maxwel Cornet': 1,
 'Kortney Hause': 1,
 'Sead Kolašinac': 2,
 'Ryan Fraser': 1,
 'Marçal': 1,
 'Adam Webster': 1,
 'Solly March': 1,
 'Stuart Dallas': 1,
 'Mohamed Elyounoussi': 1,
 'Federico Fernández': 1,
 'Jeffrey Schlupp': 1,
 'Grant Hanley': 1,
 'Nampalys Mendy': 1,
 'Jordan Ayew':

In [150]:
futbin = futbin.drop_duplicates(subset=['player_name']).reset_index()

new_names = futbin['player_name']
new_ids = futbin['player_id'].tolist()
old_ids = fix_name_ids['id'].tolist()

new_fb_names = []
new_fb_ids = []

for i in range(len(new_ids)):
    if not new_ids[i] in old_ids:
        new_fb_names.append(new_names[i])
        new_fb_ids.append(new_ids[i])

new_players = pd.DataFrame()
new_players['player_name'] = new_fb_names
new_players['player_id'] = new_fb_ids
new_players

,player_name,player_id
0,Matt Turner,233267
1,Brandon Williams,243414
2,Imran Louza,242794
3,Max Aarons,240060
4,Frank Onyeka,239529
...,...,...
326,Kevin-Prince Boateng,173909
327,Dan-Axel Zagadou,238476
328,Marwin Hitz,190243
329,Martin Hinteregger,201922


In [153]:
from rapidfuzz import process, fuzz, utils

def getMatchScore(first, second, third):
    res_strings = []
    for i in range(len(first)):
        cur = process.extract(first[i], second, scorer=fuzz.token_set_ratio, limit=1, processor=utils.default_process)
        res_strings.append([third[i], first[i], cur[0][0], cur[0][1]])

    return res_strings

In [156]:
tot_ustat = list(set(fix_name_ids.ustat_name) | set(ustat.player_name))
finds = pd.DataFrame(getMatchScore(new_players.player_name, tot_ustat, new_players.player_id), 
                     columns = ['id', 'futbin_name', 'ustat_name', 'score'])
finds.sort_values(by='score')

,id,futbin_name,ustat_name,score
317,104389,Rune Almenning Jarstein,Kevin-Prince Boateng,55.813953
27,169697,Darren Randolph,Dan Ndoye,58.333333
277,230043,Faitout Maouassa,Moussa Doumbia,60.000000
267,230727,Gautier Larsonneur,Serge Aurier,60.000000
32,204529,Michy Batshuayi,Joshua Kimmich,62.068966
...,...,...,...,...
133,227508,Melero,Gonzalo Melero,100.000000
132,228302,Alfonso Pedraza Sag,Alfonso,100.000000
131,224574,Orbelín Pineda,Orbelín Pineda,100.000000
138,204876,Carlos Clerc Martínez,Carlos Clerc,100.000000


In [157]:
finds.sort_values(by='score').to_csv("data repair csvs/check_names.csv", index=False)

## Match club to id
* Valid from FIFA 23 -> FIFA 22

In [101]:
fix_club_ids

,fb_team,futbin_id,us_team,db_id
0,Toulouse FC,1809,Toulouse,1
1,1. FC Köln,31,FC Cologne,2
2,1. FSV Mainz 05,169,Mainz 05,3
3,AC Ajaccio,614,Ajaccio,4
4,AC Monza,111811,Monza,5
...,...,...,...,...
93,VfL Wolfsburg,175,Wolfsburg,94
94,Villarreal CF,483,Villarreal,95
95,Werder Bremen,38,Werder Bremen,96
96,West Ham,19,West Ham,97


In [130]:
db_id_dict = {}

fb_t = futbin.team_title
idlist = futbin.club_id
    
for i in range(len(fbt)):
    cur = fb_t[i]
    if not cur in db_id_dict:
        db_id_dict[cur] = idlist[i]
        
#db_id_dict

In [131]:
clubs = list(db_id_dict)
new_ids = list(db_id_dict.values())
old_ids = fix_club_ids['futbin_id'].tolist()
old_ustat = fix_club_ids['us_team']

new_clubs = pd.DataFrame()

new_ustat_name = list(set(ustat.team_title.unique()).difference(old_ustat))
new_fb_name = []
new_fb_id = []

for i in range(len(new_ids)):
    if not int(new_ids[i]) in old_ids:
        new_fb_name.append(clubs[i])
        new_fb_id.append(new_ids[i])
        
new_clubs['fb_team'] = new_fb_name
new_clubs['futbin_id'] = new_fb_id

for i in range(len(new_fb_name) - len(new_ustat_name)):
    new_ustat_name.append('None')

new_clubs['us_team'] = new_ustat_name
new_clubs['db_id'] = [0 for i in range(len(new_ustat_name))]

fix_club_ids = pd.concat([fix_club_ids, new_clubs], ignore_index=True, sort=False)
fix_club_ids.to_csv('data repair csvs/fix_club_ids.csv', index=False)

In [133]:
fix_club_ids

,fb_team,futbin_id,us_team,db_id
0,Toulouse FC,1809.0,Toulouse,1.0
1,1. FC Köln,31.0,FC Cologne,2.0
2,1. FSV Mainz 05,169.0,Mainz 05,3.0
3,AC Ajaccio,614.0,Ajaccio,4.0
4,AC Monza,111811.0,Monza,5.0
...,...,...,...,...
109,FC Metz,68.0,Metz,0.0
110,ASSE,1819.0,Bordeaux,0.0
111,Bordeaux,59.0,Venezia,0.0
112,Bielefeld,159.0,None,0.0
